# ENGS106 Final Project: Deep Learning Pipeline for Optiver Data


## Import Libraries, Set Parameters, Read & Display Data

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm
import random
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 500
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.01
DROPOUT = 0.25
PATIENCE = 8
LR_PATIENCE = 5
N_FOLDS = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pd.set_option('display.max_columns', None)

# Ensures that validation groups are temporally separated from training groups by a gap (`val_group_gap`).
# Splits data into `n_splits` folds while preserving the order of groups.
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    @_deprecate_positional_args
    def __init__(self, n_splits=5, *,
                 max_train_group_size=np.inf,
                 max_val_group_size=np.inf,
                 val_group_gap=10,
                 verbose=False):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.max_val_group_size = max_val_group_size
        self.val_group_gap = val_group_gap
        self.verbose = verbose
        
    def split(self, X, y=None, groups=None):      
        X, y, groups = indexable(X, y, groups)
        n_splits = self.n_splits
        group_gap = self.val_group_gap
        max_val_group_size = self.max_val_group_size
        n_folds = n_splits + 1
        
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_groups = _num_samples(unique_groups)
        
        group_val_size = min(n_groups // n_folds, max_val_group_size)
        group_val_starts = range(n_groups - n_splits * group_val_size, n_groups, group_val_size)
        
        group_dict = {}
        n_samples = _num_samples(X)
        for idx in range(n_samples):
            group = groups[idx]
            group_dict.setdefault(group, []).append(idx)
        
        for group_val_start in group_val_starts:
            train_indices = []
            val_indices = []
            for g in unique_groups[:max(0, group_val_start - group_gap)]:
                train_indices.extend(group_dict[g])
            for g in unique_groups[group_val_start:group_val_start + group_val_size]:
                val_indices.extend(group_dict[g])
            yield train_indices, val_indices

In [9]:
def load_df_from_zip(zip_filename, csv_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zf:
        with zf.open(csv_filename) as csvfile:
            return pd.read_csv(csvfile)


df_part_1 = load_df_from_zip("data_features_part_1.zip", "data_features_part_1.csv")
# df_part_2 = load_df_from_zip("data_features_part_2.zip", "data_features_part_2.csv")
# df_part_3 = load_df_from_zip("data_features_part_3.zip", "data_features_part_3.csv")
# df_part_4 = load_df_from_zip("data_features_part_4.zip", "data_features_part_4.csv")

# Concatenate all parts into a single DataFrame
# df = pd.concat([df_part_1, df_part_2], ignore_index=True)
df = df_part_1 #[['target', 'stock_id', 'vwap', 'micro_price', 'seconds_in_bucket', 'wap', 'wap_kurtosis', 'market_urgency_v2', 'market_urgency', 'wap_skewness', 'mid_price', 'date_id', 'ask_price', 'bid_price', 'time_id', 'wap_reference_price_imb', 'wap_near_price_imb', 'seconds_in_bucket_group', 'matched_size_group_expanding_mean100', 'wap_group_first_ratio']]

display(df.head(), df.shape, df['target'].head())


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,baseline_prediction,simple_prediction,seconds_in_bucket_group,imbalance_buy_flag,imbalance_sell_flag,no_imbalance,bid_plus_ask_sizes,median_vol,high_volume,imbalance_ratio,bid_ask_volume_diff,mid_price,size_imbalance,price_spread,liquidity_imbalance,matched_imbalance,market_urgency,market_urgency_v2,far_price_reference_price_imb,near_price_reference_price_imb,near_price_far_price_imb,bid_price_reference_price_imb,bid_price_far_price_imb,bid_price_near_price_imb,ask_price_reference_price_imb,ask_price_far_price_imb,ask_price_near_price_imb,ask_price_bid_price_imb,wap_reference_price_imb,wap_far_price_imb,wap_near_price_imb,wap_bid_price_imb,wap_ask_price_imb,near_price_far_price_reference_price_imb2,bid_price_far_price_reference_price_imb2,bid_price_near_price_reference_price_imb2,bid_price_near_price_far_price_imb2,ask_price_far_price_reference_price_imb2,ask_price_near_price_reference_price_imb2,ask_price_near_price_far_price_imb2,ask_price_bid_price_reference_price_imb2,ask_price_bid_price_far_price_imb2,ask_price_bid_price_near_price_imb2,wap_far_price_reference_price_imb2,wap_near_price_reference_price_imb2,wap_near_price_far_price_imb2,wap_bid_price_reference_price_imb2,wap_bid_price_far_price_imb2,wap_bid_price_near_price_imb2,wap_ask_price_reference_price_imb2,wap_ask_price_far_price_imb2,wap_ask_price_near_price_imb2,wap_ask_price_bid_price_imb2,stock_weights,weighted_wap,wap_momentum,imbalance_momentum,spread_intensity,price_pressure,depth_pressure,spread_depth_ratio,mid_price_movement,micro_price,relative_spread,all_prices_mean,all_sizes_mean,all_prices_std,all_sizes_std,all_prices_skew,all_sizes_skew,all_prices_kurt,all_sizes_kurt,vwap,rolling_mean_matched_size,wap_skewness,wap_kurtosis,is_high_volume,matched_size_group_first_ratio,ask_price_group_first_ratio,bid_price_group_first_ratio,ask_size_group_first_ratio,bid_size_group_first_ratio,wap_group_first_ratio,near_price_group_first_ratio,far_price_group_first_ratio,reference_price_group_first_ratio,matched_size_group_expanding_mean100,ask_price_group_expanding_mean100,bid_price_group_expanding_mean100,ask_size_group_expanding_mean100,bid_size_group_expanding_mean100,wap_group_expanding_mean100,near_price_group_expanding_mean100,far_price_group_expanding_mean100,reference_price_group_expanding_mean100,matched_size_seconds_in_bucket_group_mean_ratio,ask_price_seconds_in_bucket_group_mean_ratio,bid_price_seconds_in_bucket_group_mean_ratio,ask_size_seconds_in_bucket_group_mean_ratio,bid_size_seconds_in_bucket_group_mean_ratio,wap_seconds_in_bucket_group_mean_ratio,near_price_seconds_in_bucket_group_mean_ratio,far_price_seconds_in_bucket_group_mean_ratio,reference_price_seconds_in_bucket_group_mean_ratio,matched_size_seconds_in_bucket_group_rank,ask_price_seconds_in_bucket_group_rank,bid_price_seconds_in_bucket_group_rank,ask_size_seconds_in_bucket_group_rank,bid_size_seconds_in_bucket_group_rank,wap_seconds_in_bucket_group_rank,near_price_seconds_in_bucket_group_rank,far_price_seconds_in_bucket_group_rank,reference_price_seconds_in_bucket_group_rank
0,0,0,0,3180602.69,1,0.999812,13380276.64,1.001713,0.99966,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0,0,0.1,0.0,1,0,0,69144.53,42739.16,1,0.237708,-52158.47,0.999919,7.141326,0.000214,0.754340,-0.615890,0.000161,0.000081,0.000950,-0.000076,-0.001026,0.000000,-0.000950,0.000076,0.000107,-0.000843,0.000183,0.000107,0.000094,-0.000856,0.00017,9.400884e-05,-0.000013,12.514451,1.712151e+13,0.000000,12.514451,7.882566,1.408878,4.610268,-1.927541e+12,7.882566,1.408878,9.111006,1.237706,5.039422,-1.693354e+12,9.111006,1.237706,0.138298,64.879580,0.076494,0.138298,0.004,0.004,0.0,0.0,0.0,680.648976,-107.068979,3.094966e-09,0,1.000000,0.000214,1.000170,4.157506e+06,0.000768,6.324881e+06,2.278791,1.695159,5.364051,2.775961,0.999618,13

(1309440, 130)

0   -3.029704
1   -5.519986
2   -8.389950
3   -4.010200
4   -7.349849
Name: target, dtype: float64

## Data Preparation

In [10]:
# Specify columns to scale vs id columns
id_cols = ['date_id', 'time_id', 'stock_id']
scale_cols = [col for col in df.columns if col not in id_cols + ['target']]

scaler = MinMaxScaler().fit(df[scale_cols])
def scale_data(df, scaler, scale_cols, id_cols):
    scaled = scaler.transform(df[scale_cols])
    scaled_df = pd.DataFrame(scaled, columns=scale_cols, index=df.index)
    for col in id_cols + ['target']:
        scaled_df[col] = df[col].values
    return scaled_df

df = scale_data(df, scaler, scale_cols, id_cols)
groups = df['time_id'].values

cv = PurgedGroupTimeSeriesSplit(n_splits=N_FOLDS, val_group_gap=10)


## Define Dataset, Model

In [ ]:
# - Groups data by `time_id`, treating each time step as a separate sample.
# - Sorts and reindexes stocks to ensure consistent ordering.
# - Converts selected features (`scale_cols`) and target values into PyTorch tensors.
# - Returns transposed feature tensor and target tensor for each time step.
class StockDataset(Dataset):
    def __init__(self, df, scale_cols, id_cols):
        self.scale_cols = scale_cols
        self.id_cols = id_cols
        self.groups = [group for _, group in df.groupby('time_id')]
        
    def __len__(self):
        return len(self.groups)
    
    def __getitem__(self, idx):
        group = self.groups[idx].copy()
        group = group.sort_values('stock_id')
        group = group.set_index('stock_id').reindex(range(200), fill_value=0).reset_index()
        features_tensor = torch.tensor(group[self.scale_cols].values, dtype=torch.float32)
        targets_tensor = torch.tensor(group['target'].values, dtype=torch.float32)
        return features_tensor.T, targets_tensor

class OptiverCNN(nn.Module):
    def __init__(self, input_dim):
        super(OptiverCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.AvgPool1d(2)
        
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.AvgPool1d(2)
        
        self.conv5 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(DROPOUT)
        self.fc1 = nn.Linear(256, 32)
        self.fc2 = nn.Linear(32, 200)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.relu(self.conv5(x))
        x = self.relu(self.conv6(x))
        x = self.relu(self.conv7(x))
        x = self.global_pool(x)
        x = x.squeeze(-1)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        output = self.fc2(x)
        return output

model_input_dim = len(scale_cols)


## Main Training Loop

In [ ]:
def train_fold(train_df, val_df):
    # Create dataset & data loaders for train & val
    train_ds = StockDataset(train_df, scale_cols, id_cols)
    val_ds = StockDataset(val_df, scale_cols, id_cols)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
    
    # Initialize model, loss func, optimizer, lr scheduler
    model = OptiverCNN(input_dim=model_input_dim).to(device)
    
    mae_criterion = nn.L1Loss()
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=LR_PATIENCE, factor=0.1, verbose=True)
    
    # Track best val loss and early stopping
    best_val_loss = float('inf')
    patience_counter = PATIENCE
    best_model_state = None
    
    # Training loop
    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        for X, y in tqdm(train_loader, desc=f'Fold Training Epoch {epoch+1}', leave=False):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = mae_criterion(outputs, y)
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item() * X.size(0)
        train_loss /= len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                loss = mae_criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
        val_loss /= len(val_loader.dataset)
        
        # Adjust lr based on val loss
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}: Train MAE: {train_loss:.4f} | Val MAE: {val_loss:.4f} | LR: {current_lr:.6f}")
        
        # Check for best model, implement early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = PATIENCE
            print(f"--> New best model saved with val MAE: {best_val_loss:.4f}")
        else:
            patience_counter -= 1
            if patience_counter == 0:
                print("Early stopping triggered.")
                break

    return best_model_state, best_val_loss

# CV setup
fold_results = []
df = df.reset_index(drop=True)
cv = PurgedGroupTimeSeriesSplit(n_splits=N_FOLDS, val_group_gap=10)

# Training for each fold
for fold, (train_idx, val_idx) in enumerate(cv.split(df, groups=groups)):
    print(f"\n------------------ Fold {fold+1} ------------------")
    train_fold_df = df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = df.iloc[val_idx].reset_index(drop=True)
    
    best_state, best_loss = train_fold(train_fold_df, val_fold_df)
    fold_results.append(best_loss)
    
    torch.save(best_state, f'best_cnn_model_fold_{fold+1}.pth')

# Display results
print("\nCV Results:")
print(f"Average validation loss: {np.mean(fold_results):.4f}")


------------------ Fold 1 ------------------


/Users/dawsonhaddox/anaconda3/envs/cosc78/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1: Train MAE: 5.0436 | Val MAE: 5.1468 | LR: 0.000100
--> New best model saved with val MAE: 5.1468


Epoch 2: Train MAE: 5.0413 | Val MAE: 5.1459 | LR: 0.000100
--> New best model saved with val MAE: 5.1459


Epoch 3: Train MAE: 5.0403 | Val MAE: 5.1456 | LR: 0.000100
--> New best model saved with val MAE: 5.1456


...

Epoch 6: Train MAE: 5.6854 | Val MAE: 7.2282 | LR: 0.000100


Epoch 7: Train MAE: 5.6850 | Val MAE: 7.2284 | LR: 0.000010


Epoch 8: Train MAE: 5.6845 | Val MAE: 7.2287 | LR: 0.000010


Epoch 9: Train MAE: 5.6845 | Val MAE: 7.2287 | LR: 0.000010


Epoch 10: Train MAE: 5.6845 | Val MAE: 7.2288 | LR: 0.000010
Early stopping triggered.

CV Results:
Average validation loss: 6.1233
